In [1]:
# script pion.py hjyf
from tkinter import *
import random

import numpy as np

In [2]:

    
def initialize():
    global PosX,PosY,cost, globalcost
# position initiale du robot
    PosX = 20+10*zoom
    PosY = 20+10*zoom
    for k in range(6):
        cost[k]=0
    
# cout et affichage
    Canevas.coords(Pion,PosX -9*zoom, PosY -9*zoom, PosX +9*zoom, PosY +9*zoom)
    w.config(text='Cost = '+ str(globalcost))

def draw_but(nblignes,nbcolonnes):
    
    i = random.randint(2, nblignes)
    j = random.randint(2, nbcolonnes)
    return i,j

# specification des proportion de murs, case blanches et pts de couleur
def colordraw(g,nblignes,nbcolonnes):
    pmur=0.15 
    pblanc=0.45 
    pverte=0.1
    pbleue=0.1
    prouge=0.1
  #  pnoire=0.1 mais pas besoin de le specifier c'est la couleur restante
    for i in range(nblignes):
        for j in range(nbcolonnes):
            z=np.random.uniform(0,1)
            if z < pmur:
                c=-1
            else:
                if z < pmur+ pblanc:
                    c=0
                else:    
                    if z < pmur+ pblanc + pverte:
                        c=1
                    else:
                        if z < pmur+ pblanc +pverte + pbleue:
                            c=2
                        else:
                            if z< pmur + pblanc + pverte + pbleue +prouge:
                                c=3
                            else:
                                c=4   
            g[i,j]=c
    g[0,0]=0
    g[0,1]=0
    g[2,0]=0    
    g[nblignes-1,nbcolonnes-1]=0
    g[nblignes-2,nbcolonnes-1]=0
    g[nblignes-1,nbcolonnes-2]=0
    
    for i in range(nblignes):
        for j in range(nbcolonnes):          
            y =zoom*20*i+20
            x =zoom*20*j+20
            if g[i,j]>0:            
                Canevas.create_oval(x+zoom*(10-3),y+zoom*(10-3),x+zoom*(10+3),y+zoom*(10+3),width=1,outline=color[g[i,j]],fill=color[g[i,j]])
            else:
                if g[i,j]<0:
                    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=myblack)
                    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=myblack)
    #ajoute
    i,j = draw_but(nblignes,nbcolonnes)
    g[i,j]= 5
    y =zoom*20*i+20
    x =zoom*20*j+20
    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=mygreen)
    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=mygreen)

  
def Clavier(event):
    global PosX,PosY,cost,g, globalcost
    touche = event.keysym
    cj=int((PosX-30)/(20*zoom))
    li=int((PosY-30)/(20*zoom))
    changed=0
    # deplacement aleatoire en appuyant sur space
    if touche == 'space':
        t=np.random.randint(6)
        lettre = ['f','g','h','j','y','u',]
        touche=lettre[t]
    # deplacement (-2,1)
    if touche == 'y' and li>1 and cj < nbcolonnes-1 and g[li-2,cj+1]>-1:
        PosY -= zoom*20*2
        PosX += zoom*20 
        cost[g[li-2,cj+1]]+=1 
        changed=1
    # deplacement (-2,-1)
    if touche == 't' and li>1 and cj > 0 and g[li-2,cj-1]>-1:
        PosY -= zoom*20*2       
        PosX -= zoom*20                 
        cost[g[li-2,cj-1]]+=1
        changed=1
   # deplacement (-1,2)
    if touche == 'u' and li>0 and cj < nbcolonnes-2 and g[li-1,cj+2]>-1:
        PosY -= zoom*20        
        PosX += zoom*20*2       
        cost[g[li-1,cj+2]]+=1
        changed=1
    # deplacement (-1,-2)
    if touche == 'r' and li>0 and cj >1 and g[li-1,cj-2]>-1:
        PosY -= zoom*20
        PosX -= zoom*20*2           
        cost[g[li-1,cj-2]]+=1
        changed=1
     # deplacement (2,1)  
    if touche == 'h' and li<nblignes-2 and cj < nbcolonnes-1 and g[li+2,cj+1]>-1:
        PosY += zoom*20*2
        PosX += zoom*20 
        cost[g[li+2,cj+1]]+=1
        changed=1
    # deplacement (2,-1)
    if touche == 'g' and li<nblignes-2 and cj > 0 and g[li+2,cj-1]>-1:
        PosY += zoom*20*2       
        PosX -= zoom*20                 
        cost[g[li+2,cj-1]]+=1
        changed=1
   # deplacement (1,2)
    if touche == 'j' and li<nblignes-1 and cj < nbcolonnes-2 and g[li+1,cj+2]>-1:
        PosY += zoom*20        
        PosX += zoom*20*2       
        cost[g[li+1,cj+2]]+=1
        changed=1
    # deplacement (1,-2)
    if touche == 'f' and li<nblignes-1 and cj >1 and g[li+1,cj-2]>-1:
        PosY += zoom*20
        PosX -= zoom*20*2           
        cost[g[li+1,cj-2]]+=1 
        changed=1


# La variable alea =1 si on veut des effets aleatoires sinon les transitions sont deterministes
    #On ajoute un effet aleatoire dans les transitions
    if alea==1 and changed==1:
        t=np.random.uniform(0,1)    
        if t>0.5:
            d=np.random.randint(8)
            dli=0
            if d== 0 or d==1 or d==2:
                dli=-1
            if d== 4 or d==5 or d==6:
                dli==1
            dcj=0
            if d==0 or d==7 or d==6:
                dcj=-1
            if d==2 or d==3 or d==4:
                dcj=1    
        # l'effet aleatoire est applique s'il cree un deplacement sur une case admissible     
            NewPosY = PosY+zoom*20*dli
            NewPosX = PosX+zoom*20*dcj        
            newcj=int((NewPosX-30)/(20*zoom))
            newli=int((NewPosY-30)/(20*zoom))   
            print('d',dli,dcj)
            if newli>=0 and newcj>=0 and newli<=nblignes-1 and newcj<=nbcolonnes-1 and g[newli,newcj]>-1:
                PosY=NewPosY
                PosX=NewPosX            
            
# on dessine le pion a sa nouvelle position
    Canevas.coords(Pion,PosX -9*zoom, PosY -9*zoom, PosX +9*zoom, PosY +9*zoom)       
    globalcost=0    
    for k in range(6):
        globalcost+=cost[k]*weight[k]
    w.config(text='Cost = '+ str(globalcost))

In [3]:
Mafenetre = Tk() 
Mafenetre.title('MDP')

zoom=2

alea = 0 #transitions aleatoires si alea =1 sinon mettre alea=0

#taille de la grille
nblignes=10
nbcolonnes=20 

globalcost=0

# Creation d'un widget Canvas (pour la grille)
Largeur = zoom*20*nbcolonnes+40
Hauteur = zoom*20*nblignes+40
 
# valeurs de la grille
g= np.zeros((nblignes,nbcolonnes),dtype=np.int)
cost= np.zeros(6,dtype=np.int)
weight= np.zeros(6,dtype=np.int)
weight[0] = 1
weight[1] = 10
weight[2] = 20
weight[3] = 30
weight[4] = 40
weight[5] = -1000

# def des couleurs
myred="#D20B18"
mygreen="#25A531"
myblue="#0B79F7"
mygrey="#E8E8EB"
myyellow="#F9FB70"
myblack="#2D2B2B"
mywalls="#5E5E64"
mywhite="#FFFFFF"
color=[mywhite,mygreen,myblue,myred,myblack]

# ecriture du quadrillage et coloration
Canevas = Canvas(Mafenetre, width = Largeur, height =Hauteur, bg =mywhite)
for i in range(nblignes+1):
    ni=zoom*20*i+20
    Canevas.create_line(20, ni, Largeur-20,ni)
for j in range(nbcolonnes+1):
    nj=zoom*20*j+20
    Canevas.create_line(nj, 20, nj, Hauteur-20)
colordraw(g,nblignes,nbcolonnes)

 
Canevas.focus_set()
Canevas.bind('<Key>',Clavier)
Canevas.pack(padx =5, pady =5)

PosX = 20+10*zoom
PosY = 20+10*zoom

# Creation d'un widget Button (bouton Quitter)
Button(Mafenetre, text ='Restart', command = initialize).pack(side=LEFT,padx=5,pady=5)
Button(Mafenetre, text ='Quit', command = Mafenetre.destroy).pack(side=LEFT,padx=5,pady=5)

w = Label(Mafenetre, text='Cost = '+str(globalcost),fg=myblack,font = "Verdana 14 bold")
w.pack() 

Pion = Canevas.create_oval(PosX-10,PosY-10,PosX+10,PosY+10,width=2,outline='black',fill=myyellow)

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_15168/3885692804.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  g= np.zeros((nblignes,nbcolonnes),dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_15168/3885692804.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to rev

In [4]:
class state:
    
    

    def __init__(self, coords, Value, reward, dictNStates, dictAroundStates, isReward):
        
        self.coords = coords
        self.value = Value
        self.reward = reward
        self.bestAction = ' '
        self.IsaReward = isReward
        self.dictNextStates = dictNStates
        self.nbNextStates = len(self.dictNextStates)
        self.dictAroundStates = dictAroundStates
        self.nbStatesAround = len(dictAroundStates)
        self.Probability = 1 - self.nbStatesAround/16 

    
        
    def print_state(self):

        print("coord: ",self.coords)
        print("value: ",self.value)
        print("reward: ",self.reward)
        print("dictNStates: ",self.dictNextStates)
        print("dictAroundStates: ",self.dictAroundStates)
        print("nbStatesAround: ",self.nbStatesAround)

In [5]:
class env:
    def __init__(self, nbcolonnes, nblignes, g, weight):
        self.nbcolonnes = nbcolonnes
        self.nblignes = nblignes
        self.g = g 
        self.weight = weight
        self.listAllStates = self.create_all_states()
        
    
    def create_all_states(self):
        l = list()
        for i in range(self.nblignes):
            for j in range(self.nbcolonnes):
                coord =  (i,j)
                isaReward = False
                if g[i,j] < 0:
                    reward  = " mur "
                else:
                    #reward = np.format_float_scientific(self.weight[g[i,j]],exp_digits=1)
                    reward = -self.weight[g[i,j]]
                    if reward > 0: 
                        isaReward = True
                value = 0
                dictNStates = self.get_dictNextStates(i, j)
                dictAroundStates = self.get_around_States_Proba(i, j)
                
                
                s = state(coord, value, reward, dictNStates, dictAroundStates,isaReward )
                l.append(s)
        return l
    def get_dictNextStates(self,i,j):
        #check OutOfBound
        di = dict()
        if i-2>=     0       and j+1 <self.nbcolonnes and self.g[i-2,j+1] != -1:
            di['y']=(i-2,j+1)
        if i-1>=     0       and j+2 <self.nbcolonnes and self.g[i-1,j+2] != -1:
            di['u']=(i-1,j+2)
        if i+1<self.nblignes and j+2 <self.nbcolonnes and self.g[i+1,j+2] != -1:
            di['j']=(i+1,j+2)
        if i+2<self.nblignes and j+1 <self.nbcolonnes and self.g[i+2,j+1] != -1:
            di['h']=(i+2,j+1)
        if i+2<self.nblignes and j-1 >=       0       and self.g[i+2,j-1] != -1:
            di['g']=(i+2,j-1)
        if i+1<self.nblignes and j-2 >=       0       and self.g[i+1,j-2] != -1:
            di['f']=(i+1,j-2)
        if i-1>=     0       and j-2 >=       0       and self.g[i-1,j-2] != -1:
            di['r']=(i-1,j-2)
        if i-2>=     0       and j-1 >=       0       and self.g[i-2,j-1] != -1:
            di['t']=(i-2,j-1)

        return di
    def get_around_States_Proba(self, i, j):
        # check OutOfBound
        diStateAround = dict()
        if i+1<self.nblignes and j   <self.nbcolonnes and self.g[i+1,j  ] != -1:
            diStateAround['md'] = (i+1,j  , 1/16)

        if i-1>=      0      and j   <self.nbcolonnes and self.g[i-1,j  ] != -1:
            diStateAround['mg'] = (i-1,j  , 1/16)

        if i+1<self.nblignes and j+1 <self.nbcolonnes and self.g[i+1,j+1] != -1:
            diStateAround['hd'] = (i+1,j+1, 1/16)

        if i-1>=      0      and j-1 >=      0        and self.g[i-1,j-1] != -1:
            diStateAround['bg'] = (i-1,j-1, 1/16)

        if i+1<self.nblignes and j-1 >=      0        and self.g[i+1,j-1] != -1:
            diStateAround['bd'] = (i+1,j-1, 1/16)

        if i-1>=      0      and j+1 <self.nbcolonnes and self.g[i-1,j+1] != -1:
            diStateAround['hg'] = (i-1,j+1, 1/16)

        if i <self.nblignes  and j+1 <self.nbcolonnes and self.g[i  ,j+1] != -1:
            diStateAround['hm'] = (i  ,j+1, 1/16)

        if i <self.nblignes  and j-1 >=      0        and self.g[i  ,j-1] != -1:
            diStateAround['bm'] = (i  ,j-1, 1/16)
            
        return diStateAround

    def get_state(self,i,j):
        i = int(i)
        j = int(j)
        if i>=0 and i<self.nblignes and j >=0 and j<self.nbcolonnes:
            return self.listAllStates[i*self.nbcolonnes+j]
        return None

    def afficheV(self):
        print("Valeur de l'etat selon l'algo")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                value="{:.2e}".format(self.get_state(i,j).value)
                l.append(value)
            print(l)
          

    def afficheBestAction(self):
        print("meilleure Action")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).bestAction)
            print(l)

    def afficheP(self):
        print("Proba d'arriver ce State")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).Probability)
            print(l)

    def afficheNBSA(self):
        print("NB State Around")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).nbStatesAround)
            print(l)

    def afficheR(self):
        print("Rewards")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).reward)
            print(l)
    def afficheNBNS(self):
        print("NB Next State ")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append((self.get_state(i,j).nbNextStates,self.get_state(i,j).dictNextStates))
            print(l)



In [6]:
e = env(nbcolonnes,nblignes,g,weight=weight)
    
e.afficheV()
e.afficheP()
e.afficheNBSA()
e.afficheR()
e.afficheBestAction()
e.afficheNBNS() 

Valeur de l'etat selon l'algo
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00

In [7]:






def value_iterationG(nbcolonnes,nblignes, gamma, epsilon):
    Value_states = np.zeros(nblignes*nbcolonnes, dtype=np.int)
    e = env(nbcolonnes,nblignes,g,weight=weight)
    
    e.afficheV()
    e.afficheP()
    e.afficheP()
    # actions = ['f','g','h','j','y','u',]
    iteration = 0
    stop = False
    while stop==False:
        
        for count, s in enumerate(e.listAllStates):
            best = float('-inf')
            saveA = ' '
            
            if s.reward == " mur ":
                continue
            elif s.IsaReward == True:
                
                s.bestAction = 'R'
                continue
            for a, v in s.dictNextStates.items():
                somme = 0
                pState = e.get_state(v[0],v[1]) 
                somme = pState.Probability*pState.value 
                #print(pState.Probability)             
                for cState in pState.dictAroundStates.values():
                    
                    sA = e.get_state(cState[0],cState[1])
                    somme+= (1/16*sA.value)
                        
                temp = s.reward+gamma*somme

                if temp > best:
                    best = temp
                    saveA = a
                
                
            s.bestAction = saveA
            s.value = best
            # print(tempBest)
            # for i in tempBest :
            #     bestV = 0
            #     bestA = 'no'
            #     if i[1]>bestV :
            #         bestV = i[1]
            #         bestA = i[0]

            # s.value = bestV
            # s.bestAction = bestA
            
          
            
            
        iteration +=1
       

        if iteration>1000:
            stop = True
            e.afficheV()
            e.afficheBestAction()
            e.afficheR()
            print(iteration)
    

gamma = 0.9
epsilon = 1

value_iterationG(nbcolonnes=nbcolonnes,nblignes=nblignes, gamma=gamma, epsilon=epsilon)
    

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_15168/4198120505.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Value_states = np.zeros(nblignes*nbcolonnes, dtype=np.int)


Valeur de l'etat selon l'algo
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00

In [8]:


initialize()

Mafenetre.mainloop()

